In [1]:
import pathlib
import jinja2
from xml.dom import minidom
from tqdm import tqdm
from IPython.display import HTML
import re
import html


In [9]:
def close_enough(match, limit = 0.01):
    f = float(match.group())
    i = round(f) 
    if abs(f - i) < limit:
        return str(i)
    else: 
        return str(f)
    
re.sub(r'\d*\.\d+', close_enough, '0.1 0.001 3.999 3.98')

'0.1 0 4 3.98'

In [14]:
def clean_svg(path):
    svg_keep = {'viewBox'}
    dom = minidom.parse(str(path)).documentElement
    clean_attrs(dom, None, {'viewBox'})
    clean_children(dom)
    flatten(dom)
    return dom


def flatten(element):
    kill = []
    for child in list(element.childNodes)[::-1]:
        if child.nodeName == 'g':
            flatten(child)
            for subchild in list(child.childNodes):
                clone = child.removeChild(subchild)
                element.insertBefore(clone, child)
            element.removeChild(child)
            child.unlink()
                

def clean_children(element):
    element_remove = {'stroke', 'style', 'stroke-linejoin', 'stroke-miterlimit', 'fill', 'stroke-width'}
    for child in list(element.childNodes):
        if child.nodeType != child.ELEMENT_NODE:
            element.removeChild(child)
        else:
            clean_attrs(child, element_remove)
            clean_children(child)
        
def clean_attrs(child, remove, keep=None):
    for attr in list(child.attributes.keys()):
        if remove and attr in remove:
            child.attributes.removeNamedItem(attr)
        elif keep and not attr in keep:
            child.attributes.removeNamedItem(attr)
        else:
            attr = child.attributes.get(attr)
            value = re.sub(r' +', ' ', attr.value.strip())
            value = re.sub(r'\d*\.\d+', close_enough, value)
            attr.value = value
        
def pascal_case(name):
    return re.sub(r'[_\W]', '', name.title())

TEMPLATE = """
import React from 'react'

const {{ name }} = () => (
  {{ xml|indent(2) }}
)
export default {{ name }}
"""
react_template = jinja2.Template(TEMPLATE.strip())

    
    
def make_html_element(name, svg_node):
    svg_node.attributes['id'] = name
    svg_node.attributes['class'] = 'icon'
    xml = svg_node.toprettyxml('  ')
    name = pascal_case(name)
    return '<span class=icon> {1}<div class=popup><h2>{0}</h2><pre><code>{2}</code></pre></div></span>'.format(
        name, xml, html.escape(xml))

def make_react_component(name, svg_node, writedir):
    component_name = pascal_case(name)
    svg_node.attributes['className'] = 'Icon'
    svg_node.attributes['id'] = component_name
    xml = svg_node.toprettyxml('  ').replace('/>', ' />')
    result = react_template.render(name=component_name, xml=xml) 
    component_path = (writedir / component_name).with_suffix('.js')
    component_path.write_text(result + '\n')
    
def make_svg_file(name, svg_node, writedir):
    svg_node.attributes['style'] = 'fill:none; stroke:black; stroke-width:2;'
    xml_header = '<?xml version="1.0" encoding="utf-8"?>'
    content = '\n'.join([xml_header, svg_node.toprettyxml('  ')])
    writeto = (writedir / name).with_suffix('.svg')
    writeto.write_text(content)

def clean_files(readdir, n=None):
    files = sorted([fn for fn in readdir.iterdir() if fn.suffix == '.svg'])[:n]
    return ((fn.stem, clean_svg(fn)) for fn in tqdm(files))

def main(readdir, output_format, n=None):
    svgs = clean_files(readdir, n)
    
    if output_format.upper() == 'SVG':
        writedir = pathlib.Path('.') / 'icons2'
        writedir.mkdir(exist_ok=True)
        for name, svg_node in svgs:
            make_svg_file(name, svg_node, writedir)

    elif output_format.upper() == 'HTML':
        styles = (
            '<style>'
            'svg { fill:none; stroke:black; stroke-width:2;width: 3em; }'
            '.icon { position: relative }'
            '.popup { '
                'width: 30em; opacity: 0; background: white; z-index: 10000; '
                'position: fixed; top: 10em; left: 1em; border: 2px solid; '
                'background: white; padding: 10px; pointer-events: none;' 
            '}'
            'code, pre { background: none; }'
            '.icon:hover .popup { opacity: 0.8 }'
            '</style>'
        )
        
        html_header = styles
        svg_nodes = [make_html_element(name, svg_node) for name, svg_node in svgs]
        html = html_header + ''.join(svg_nodes)
        return HTML(html)
    
    elif output_format.upper() == 'REACT':
        writedir = pathlib.Path('.') / '..' / 'components' / 'icons' / 'lineicons'
        #writedir = pathlib.Path('.') / 'components'
        writedir.mkdir(exist_ok=True)
        for name, svg_node in svgs:
            make_react_component(name, svg_node, writedir)
    
    else: 
        output = '' 
        for name, svg_node in svgs:
            output += '\n'.join([name, svg_node.toprettyxml('  '), ''])
        print(output)


    
    
#readdir = pathlib.Path.home() / 'Desktop' / 'icons'
readdir = pathlib.Path('./icons')
main(readdir, 'svg')



  0%|          | 0/715 [00:00<?, ?it/s]

  5%|▍         | 33/715 [00:00<00:02, 325.13it/s]

  9%|▊         | 62/715 [00:00<00:02, 309.55it/s]

 13%|█▎        | 91/715 [00:00<00:02, 303.26it/s]

 17%|█▋        | 125/715 [00:00<00:01, 310.97it/s]

 22%|██▏       | 158/715 [00:00<00:01, 310.84it/s]

 26%|██▌       | 186/715 [00:00<00:01, 299.23it/s]

 30%|██▉       | 213/715 [00:00<00:01, 273.05it/s]

 33%|███▎      | 239/715 [00:00<00:01, 250.27it/s]

 37%|███▋      | 264/715 [00:00<00:01, 247.18it/s]

 40%|████      | 288/715 [00:01<00:01, 222.23it/s]

 43%|████▎     | 311/715 [00:01<00:02, 194.76it/s]

 46%|████▋     | 332/715 [00:01<00:02, 182.77it/s]

 49%|████▉     | 351/715 [00:01<00:02, 157.72it/s]

 51%|█████▏    | 368/715 [00:01<00:02, 158.39it/s]

 54%|█████▍    | 385/715 [00:01<00:02, 151.79it/s]

 56%|█████▌    | 401/715 [00:01<00:02, 149.73it/s]

 58%|█████▊    | 417/715 [00:01<00:02, 146.64it/s]

 61%|██████▏   | 439/715 [00:02<00:01, 161.90it/s]

 64%|██████▍   | 459/715

In [4]:
%debug

ERROR: No traceback has been produced, nothing to debug.


# import html
s ='<code></code>'
#c = s.encode('ascii', 'xmlcharrefreplace')
html.escape(s)


In [5]:
element = data[2][1]
for child in element.childNodes:
    print(child.nodeName)

NameError: name 'data' is not defined

In [ ]:
def significant_digits(num, digits=3):
    return '{num:.{digits}f}'.format(num=float(num), digits=digits)

significant_digits('213.23')

In [ ]:
import re
data = {}
for path in dom.getElementsByTagName('path'):
    path_d = path.attributes['d'].value
    icon_id = path.attributes['id'].value
    path_d = re.sub(r'-?\d+\.\d+', lambda m: significant_digits(m.group(), 3), path_d)
    data[icon_id.lower()] = path_d

data

In [ ]:
icon_id.value

In [ ]:
ldir = pathlib.Path('..') / 'components' / 'lineicons'
icons = [i.stem for i in ldir.iterdir()]
exports = '\n'.join('export {{ {0} }} from "./{0}.js"'.format(icon) for icon in icons)
index = ldir / 'index.js'
index.write_text(exports)